**First approach on using modules (Pyrocko, MTFIT) to convert moment tensor components and strike, dip, rake angles**

# Moment tensor to strike, dip and rake

## 1. Pyrocko

In [4]:
from pyrocko import moment_tensor as pmt
import numpy as num

r2d = 180. / num.pi

magnitude = 6.3  # Magnitude of the earthquake

exp = pmt.magnitude_to_moment(magnitude)  # convert the mag to moment in [Nm]

# init pyrocko moment tensor
m = pmt.MomentTensor(
    mnn=2.34*exp,
    mee=-2.64*exp,
    mdd=0.295*exp,
    mne=1.49*exp,
    mnd=0.182*exp,
    med=-0.975*exp)

print(m)  # print moment tensor

# gives out both nodal planes:
(s1, d1, r1), (s2, d2, r2) = m.both_strike_dip_rake()

print('strike1=%g, dip1=%g, rake1=%g' % (s1, d1, r1))
print('strike2=%g, dip2=%g, rake2=%g' % (s2, d2, r2))


# p-axis normal vector in north-east-down coordinates
p_ned = m.p_axis()

print('p_ned=(%g, %g, %g)' % tuple(p_ned))

# convert to azimuth and dip
p_azimuth = num.arctan2(p_ned[1], p_ned[0]) * r2d
p_dip = num.arcsin(p_ned[2]) * r2d

print('p_azimuth=%g, p_dip=%g' % (p_azimuth, p_dip))

Scalar Moment [Nm]: M0 = 9.73137e+18 (Mw = 6.6)
Moment Tensor [Nm]: Mnn =  0.740,  Mee = -0.835, Mdd =  0.093,
                    Mne =  0.471,  Mnd =  0.058, Med = -0.308    [ x 1e+19 ]
Fault plane 1 [deg]: strike = 242, dip =  78, slip-rake = -171
Fault plane 2 [deg]: strike = 150, dip =  81, slip-rake =  -12

strike1=241.541, dip1=77.736, rake1=-170.55
strike2=149.516, dip2=80.7675, rake2=-12.4275
p_ned=(-0.253411, 0.930737, 0.26365)
p_azimuth=105.231, p_dip=15.2868


## 2. MTFit

In [5]:
import numpy as np
from MTfit.convert.moment_tensor_conversion import MT33_SDR

# Define your 3x3 moment tensor (example values)
moment_tensor = np.array([[1, 1, 1],
                          [1, 1, 1],
                          [1, 1, 1]])

# Use MT33_SDR to convert the moment tensor to strike, dip, and rake
# MT33_SDR returns two solutions for the two possible fault planes
fault_planes = MT33_SDR(moment_tensor)
fault_planes = np.degrees(fault_planes)
# Output the results
print(f"Fault Plane 1 - Strike: {fault_planes[0][0]:.1f}, Dip: {fault_planes[1][0]:.1f}, Rake: {fault_planes[2][0]:.1f}")
#print(f"Fault Plane 2 - Strike: {fault_planes[1][0]:.1f}, Dip: {fault_planes[1][1]:.1f}, Rake: {fault_planes[1][2]:.1f}")


Fault Plane 1 - Strike: 91.5, Dip: 77.0, Rake: 142.7


# Strike, dip and rake to moment tensor

## 1. Pyrocko

In [6]:
from pyrocko import moment_tensor as pmt

magnitude = 6.3  # Magnitude of the earthquake

m0 = pmt.magnitude_to_moment(magnitude)  # convert the mag to moment

strike = 130
dip = 40
rake = 110
mt = pmt.MomentTensor(strike=strike, dip=dip, rake=rake, scalar_moment=m0)

m6 = [mt.mnn, mt.mee, mt.mdd, mt.mne, mt.mnd, mt.med]  # The six MT components
print(m6/mt.scalar_moment())  # normalized MT components

[-0.75956309 -0.16585349  0.92541658 -0.4175028  -0.29341204  0.09581821]
